# Use non-human organismal FASTA file to find human homologs on NCBI BLAST.
This is a neat little program that will read in a fasta file from a non-human organism, and use the gene sequences in the fasta file to search NCBI for human homologs. 

This does require that the fasta file contain genes, however, if you modify the Human_Disease_Gene_Extraction_NCBI ipython notebook you can create your own fasta file with the coding regions of genes.

Or you could use Bioconductor.

I am going to use a fasta file from a Goat, that was given to me as an example.

#### As always, we start with our imports. We have a few more than usual but they will all come in handy. 

In [ ]:
from Bio import SeqIO
from Bio.Blast import NCBIWWW
from Bio.Blast import NCBIXML
import time
from urllib3.exceptions import HTTPError

#### This is a nice little loop that will read in our fasta file, and use the SeqIO.parse to grab all the headers and sequences from our fasta file. 
We did have to use the .split function on our header in order to append ONLY the GI number in the header

In [ ]:
header = []
sequence = []
for record in SeqIO.parse("ch_ests_18.fasta", "fasta"):
    header.append(record.id.split("|")[1])
    sequence.append(record.seq)


#### It is import to note that you can blast the GI number in the header OR the actual sequence itself in the next section. I am choosing to use the GI number in this example.

#### The meat and potatoes
Ok so here is a for loop that will use our 'header' list to blast and find human homologs (remember our fasta file contains gene sequences from a goat). So we say for h in header range 0 to 9 (python in not inclusive, as in it will start at 0 and stop at 10 but not perform a task on the 10th element).

We use the'try' logic here which is useful becasue every now and then your program will stall out if the sequence is taking too long to blast. For that reason, we will use an 'except' statement when this HTTPError occurs. Getting back to the logic in the 'try' block we start by printing which element we are blasting, this is more for a sanity check than functionality. This program takes a while to run and personally I do not like looking at a blank screen with no feedback about how my program is running. Next we will start a timer, this will come in handy later. This blast_result_handle variable utilizes the NCBIWWW.qblast function with the inputs, we want to use the blastx program, refseq protein database, and we use our list to iterate for the sequence, and lastly we use the human txid id for the entrez_query (if you wanted to blast for homologs in dogs you would change this input). You can find more info about this .qblast [here](https://www.ncbi.nlm.nih.gov/books/NBK25499/#chapter4.ESearch) and [here](http://biopython.org/DIST/docs/api/Bio.Entrez-module.html#esearch). 

Ok so the next part gets a little busy with a for loop and a with open loop that writes a file all at once! Lets take it step by step. This for loop is just grabbing the first alignment that is being returned from the search (assuming you only want the first alignment, you could change this to grab every alignment that the blast found if you really wanted). We then open a file called 'goat_human.tst' that writes to it the goat GI number, the human alignment GI number returned and the actual name of the gene. Then we take the human GI number and write that to a file (this becomes useful if you wanted to throw this into a Gene Ontology website like Panther to perform an analysis). Lastly we will close the blast_result_handle.

In [ ]:
## change your range before you run this!!!
for h in header[0:10]:    
    try:
        print('blasting', h)
        timestart = time.clock()   
        blast_result_handle = NCBIWWW.qblast(program="blastx",database="refseq_protein",sequence= h,entrez_query= "txid9606[ORGN]")   
        blast_records = NCBIXML.parse(blast_result_handle)
    
        blast_record = next(blast_records)
        print ("# alignments:",len(blast_record.alignments))
        for alignment in blast_record.alignments[0:1]:
            with open('goat_human.tsv','a') as out:
                out.write("{}\t{}\t{}\n".format(h,alignment.hit_id.split('|')[1],alignment.hit_def.split('>')[0]))
            with open('human_id.txt','a') as out:
                out.write("{}\n".format(alignment.hit_id.split('|')[1]))
        blast_result_handle.close()
        timestop = time.clock()
        print ("time",timestop-timestart)
        print('done')

    except HTTPError as e:
        time.sleep(500)
        next